In [1]:
#init
import torch, numpy, sys
sys.path.append("..")
from chatbot_end2end.module.interact_session import InteractSession
from nlptools.utils import Config
import torch.nn as nn

cfg = Config("../config/elsa.yml")
session = InteractSession.build(cfg)

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'num_hidden_layers': 3, 'num_attention_heads': 3, 'hidden_act': 'gelu', 'intermediate_size': 1024, 'hidden_dropout_prob': 0, 'attention_probs_dropout_prob': 0, 'max_position_embeddings': 512, 'type_vocab_size': 2, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12}, 'decoder': {'num_hidden_layers': 3, 'num_attention_heads': 3, 'intermediate_size': 1024, 'shared_embed': True}}
load model from file data/cornell/model_transformer.pt


In [2]:
skill_name = "cornell"
skill = session.topic_manager.skills[skill_name]
state_dict = skill.model.decoder.state_dict()
dialog_status = session.new_dialog()
dialog_status.add_utterance("I would hardly say you look like trash")
data = dialog_status.status2data()

encoder_out, encoder_hidden = skill.model.dialog_embedding(data["utterance"].data, data["utterance_mask"].data, data["sentiment"].data)
utterance_mask = data["utterance_mask"].data
print(encoder_out.shape)
print(utterance_mask.shape)

torch.Size([1, 20, 768])
torch.Size([1, 20])


In [19]:
import nlptools.zoo.encoders.transformer
import chatbot_end2end.model.generative_tracker
import importlib
importlib.reload(nlptools.zoo.encoders.transformer)
importlib.reload(chatbot_end2end.model.generative_tracker)

from nlptools.zoo.encoders.transformer import TransformerDecoder
from chatbot_end2end.model.generative_tracker import GenerativeTracker

embedding = skill.model.encoder.embedding

decoder_hidden_layers = skill.model.config["decoder"]["num_hidden_layers"]
decoder_attention_heads = skill.model.config["decoder"]["num_attention_heads"]
decoder_hidden_size = skill.model.config["decoder"]["intermediate_size"]

# bert_model_name = skill.model.config["bert_model_name"]
pad_id = skill.model.pad_id
bos_id = skill.model.bos_id
eos_id = skill.model.eos_id
unk_id = skill.model.unk_id
beam_size = 3
len_penalty = 1
unk_penalty = 1
dropout=0

shared_layers = {"encoder": skill.model.encoder}

tracker = GenerativeTracker(shared_layers=shared_layers, skill_name=skill_name, decoder_hidden_laers=decoder_hidden_layers,
                             decoder_attention_heads=decoder_attention_heads, decider_hidden_size=decoder_hidden_size, dropout=dropout, pad_id=pad_id, bos_id=bos_id,
                            eos_id=eos_id, unk_id=unk_id, beam_size=beam_size, len_penalty=len_penalty, unk_penalty=unk_penalty)
#tracker = tracker.to("cuda:0")

decoder = TransformerDecoder(embedding, decoder_hidden_layers, decoder_attention_heads, decoder_hidden_size, dropout)
#decoder = decoder.to("cuda:0")
decoder.load_state_dict(state_dict)

tracker.decoder = decoder

result, score = tracker.beam_search(encoder_out, encoder_hidden, utterance_mask)
print(result, score)
response = result[0].detach().numpy()
response = response[response>0][1:-1]
response = skill.tokenizer.tokens2sentence(skill.vocab.id2words(response))
score = score.item()
print(response)

model config {'encoder': {'vocab_size': 30522, 'hidden_size': 768, 'num_hidden_layers': 3, 'num_attention_heads': 3, 'hidden_act': 'gelu', 'intermediate_size': 1024, 'hidden_dropout_prob': 0, 'attention_probs_dropout_prob': 0, 'max_position_embeddings': 512, 'type_vocab_size': 2, 'initializer_range': 0.02, 'layer_norm_eps': 1e-12}, 'decoder': {'num_hidden_layers': 1, 'num_attention_heads': 3, 'intermediate_size': 1024, 'shared_embed': True}}
tensor([[ 101, 2053, 4283, 2000, 2017, 1012,  102,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0]]) [-0.03192614]
no thanks to you .
